In [20]:
import pandas as pd
import numpy as np

In [21]:
data=pd.read_csv("/content/drive/MyDrive/tested.csv")

In [22]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [24]:
data.shape

(418, 12)

In [25]:
data=data[['PassengerId','Survived', 'Pclass','Sex','Age','Fare','Embarked']]
data

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked
0,892,0,3,male,34.5,7.8292,Q
1,893,1,3,female,47.0,7.0000,S
2,894,0,2,male,62.0,9.6875,Q
3,895,0,3,male,27.0,8.6625,S
4,896,1,3,female,22.0,12.2875,S
...,...,...,...,...,...,...,...
413,1305,0,3,male,NaN,8.0500,S
414,1306,1,1,female,39.0,108.9000,C
415,1307,0,3,male,38.5,7.2500,S
416,1308,0,3,male,NaN,8.0500,S


In [26]:
data.corr()['Survived']

<ipython-input-26-fdffd623bf21>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()['Survived']


PassengerId   -0.023245
Survived       1.000000
Pclass        -0.108615
Age           -0.000013
Fare           0.191514
Name: Survived, dtype: float64

In [27]:
#data['PassengerId'].isnull().any()
#data['Survived'].isnull().any()
#data['Age'].isnull().any()  #age has null values
#data['Fare'].isnull().any()  #Fare has null values
#data['Pclass'].isnull().any()
data['Embarked'].isnull().any()

average_age= data['Age'].mean()
average_fare= data['Fare'].mean()

data['Age'].fillna(average_age, inplace=True)
data['Fare'].fillna(average_fare, inplace=True)



In [28]:
data.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age            False
Fare           False
Embarked       False
dtype: bool

In [29]:
import xgboost as xgb

model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, random_state=42)

In [30]:
y=data['Survived']
X=data.drop('Survived',axis=1)

In [31]:
y

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [32]:
X

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,male,34.50000,7.8292,Q
1,893,3,female,47.00000,7.0000,S
2,894,2,male,62.00000,9.6875,Q
3,895,3,male,27.00000,8.6625,S
4,896,3,female,22.00000,12.2875,S
...,...,...,...,...,...,...
413,1305,3,male,30.27259,8.0500,S
414,1306,1,female,39.00000,108.9000,C
415,1307,3,male,38.50000,7.2500,S
416,1308,3,male,30.27259,8.0500,S


In [39]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X['Sex'] = ordinal_encoder.fit_transform(X[['Sex']])
X['Embarked'] = ordinal_encoder.fit_transform(X[['Embarked']])

In [40]:
X

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,1.0,34.50000,7.8292,1.0
1,893,3,0.0,47.00000,7.0000,2.0
2,894,2,1.0,62.00000,9.6875,1.0
3,895,3,1.0,27.00000,8.6625,2.0
4,896,3,0.0,22.00000,12.2875,2.0
...,...,...,...,...,...,...
413,1305,3,1.0,30.27259,8.0500,2.0
414,1306,1,0.0,39.00000,108.9000,0.0
415,1307,3,1.0,38.50000,7.2500,2.0
416,1308,3,1.0,30.27259,8.0500,2.0


In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [42]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [43]:
y_pred = model.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score

In [45]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

#As the dataset is very simple and small, the accuracy is 100%

Accuracy: 1.0
